In [11]:
#!pip install torch

In [12]:
#!pip install --upgrade torch

In [13]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import variable

In [14]:
col = ['Id','Title','Genres']
movies = pd.read_csv('ml-1m/movies.dat',sep='::',header=None,names=col,engine='python',encoding='latin-1')
movies.head()

,Id,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
#col = ['UserId','Gender','Age','UserJob','ZipCode']
users = pd.read_csv('ml-1m/users.dat',sep='::',header=None,engine='python',encoding='latin-1')
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [18]:
rating = pd.read_csv('ml-1m/ratings.dat',sep='::',header=None,engine='python',encoding='latin-1')
rating.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [24]:
rating.shape

(1000209, 4)

In [21]:
training_set = pd.read_csv(filepath_or_buffer='ml-100k/u1.base',sep='\t',header=None)
training_set.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [22]:
training_set.shape

(80000, 4)

In [25]:
training_set = np.array(training_set,dtype='int')

In [26]:
test_set = pd.read_csv(filepath_or_buffer='ml-100k/u1.test',sep='\t',header=None)
test_set.head()

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


In [27]:
test_set.shape

(20000, 4)

In [28]:
test_set = np.array(test_set,dtype='int')

In [31]:
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_users

943

In [32]:
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))
nb_movies

1682

In [35]:
training_set[0:5]

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712]])

In [36]:
def convert(data):
    new_data = []
    for id_users in range(1,nb_movies+1):
        id_movies = data[:,1][data[:,0]==id_users]
        id_rating = data[:,2][data[:,0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_rating
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)

In [37]:
training_set[0:5]

[[5.0,
  3.0,
  4.0,
  3.0,
  3.0,
  0.0,
  4.0,
  1.0,
  5.0,
  0.0,
  2.0,
  0.0,
  5.0,
  0.0,
  5.0,
  5.0,
  0.0,
  4.0,
  5.0,
  0.0,
  1.0,
  4.0,
  0.0,
  0.0,
  4.0,
  3.0,
  0.0,
  4.0,
  1.0,
  3.0,
  0.0,
  5.0,
  0.0,
  2.0,
  1.0,
  0.0,
  2.0,
  3.0,
  0.0,
  3.0,
  2.0,
  5.0,
  4.0,
  0.0,
  5.0,
  4.0,
  0.0,
  5.0,
  0.0,
  5.0,
  0.0,
  4.0,
  0.0,
  0.0,
  5.0,
  0.0,
  5.0,
  4.0,
  5.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  4.0,
  0.0,
  4.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  4.0,
  0.0,
  4.0,
  0.0,
  4.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  5.0,
  4.0,
  5.0,
  0.0,
  0.0,
  0.0,
  5.0,
  2.0,
  4.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  2.0,
  4.0,
  0.0,
  0.0,
  5.0,
  1.0,
  5.0,
  0.0,
  0.0,
  0.0,
  5.0,
  3.0,
  0.0,
  0.0,
  5.0,
  0.0,
  0.0,
  3.0,
  4.0,
  5.0,
  0.0,
  2.0,
  5.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  4.0,
  0.0,
  4.0,
  3.0,
  5.0,
  1.0,
  3.0,
  0.0,
  3.0,
  2.0,
  0.0,

In [38]:
test_set[0:5]

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198]])

In [39]:
test_set = convert(test_set)
test_set[0:5]

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  5.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  5.0,
  0.0,
  5.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  4.0,
  0.0,
  0.0,
  4.0,
  3.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  4.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  4.0,
  0.0,
  0.0,
  0.0,
  0.0,
  5.0,
  0.0,
  0.0,
  4.0,
  0.0,
  3.0,
  0.0,
  4.0,
  0.0,
  3.0,
  3.0,
  0.0,
  4.0,
  0.0,
  0.0,
  0.0,
  5.0,
  4.0,
  3.0,
  0.0,
  5.0,
  4.0,
  0.0,
  3.0,
  0.0,
  3.0,
  3.0,
  0.0,
  4.0,
  3.0,
  1.0,
  0.0,
  4.0,
  0.0,
  1.0,
  0.0,
  4.0,
  5.0,
  5.0,
  0.0,
  4.0,
  3.0,
  5.0,
  0.0,
  0.0,
  0.0,
  4.0,
  5.0,
  3.0,
  0.0,
  0.0,
  0.0,
  5.0,
  3.0,
  4.0,
  0.0,
  5.0,
  0.0,
  2.0,
  1.0,
  1.0,
  0.0,
  0.0,
  4.0,
  5.0,
  0.0,
  0.0,
  0.0,
  1.0,
  5.0,
  5.0,
  0.0,
  0.0,
  3.0,
  3.0,
  0.0,
  1.0,
  4.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  4.0,
  5.0,
  3.0,
  0.0,
  4.0,
  0.0,
  4.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,

In [40]:
training_set = torch.FloatTensor(training_set)
training_set

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [41]:
test_set = torch.FloatTensor(test_set)
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [42]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0

training_set[training_set == 3] = 1
training_set[training_set == 4] = 1
training_set[training_set == 5] = 1

In [43]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0

test_set[test_set == 3] = 1
test_set[test_set == 4] = 1
test_set[test_set == 5] = 1

In [55]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [60]:
nb_epoch = 10

for epoch in range(1,nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(0,nb_users-batch_size,batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_= rbm.sample_h(v0)
        
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0,vk,ph0,phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
        s += 1.
    print("epoch: "+ str(epoch)+" loss: "+str(train_loss/s))

epoch: 1 loss: tensor(0.2512)
epoch: 2 loss: tensor(0.2462)
epoch: 3 loss: tensor(0.2521)
epoch: 4 loss: tensor(0.2470)
epoch: 5 loss: tensor(0.2467)
epoch: 6 loss: tensor(0.2473)
epoch: 7 loss: tensor(0.2476)
epoch: 8 loss: tensor(0.2490)
epoch: 9 loss: tensor(0.2476)
epoch: 10 loss: tensor(0.2495)


In [63]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2505)
